In [4]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit import ParameterVector
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import pandas as pd

# === 1. เตรียมข้อมูล ===
X, y = load_iris(return_X_y=True)
X = X[y == 0][:50]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === 2. แปลงเป็น bitstring (แบบง่าย)
def float_to_bits(x):
    return ''.join(['1' if xi > 0.5 else '0' for xi in x])

bitstrings = [float_to_bits(x) for x in X_scaled]
n_qubits = 4

# === 3. กำหนดพารามิเตอร์ของวงจร
params = ParameterVector('θ', length=8)

def create_vqc(params):
    qc = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        qc.ry(params[i], i)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    for i in range(n_qubits):
        qc.rz(params[i + 4], i)
    qc.measure_all()
    return qc

# === 4. Loss function เทียบ distribution จริง vs วงจร
def target_distribution(bitstrings):
    counts = Counter(bitstrings)
    total = sum(counts.values())
    dist = np.zeros(2**n_qubits)
    for bs, c in counts.items():
        idx = int(bs, 2)
        dist[idx] = c / total
    return dist

target_probs = target_distribution(bitstrings)

# === 5. แปลงพารามิเตอร์จาก vector → float
def evaluate_circuit(theta_values, shots=1000):
    param_dict = dict(zip(params, theta_values))
    qc = create_vqc(params).assign_parameters(param_dict)
    backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    probs = np.zeros(2**n_qubits)
    for b, c in counts.items():
        b = b[::-1]  # flip
        idx = int(b, 2)
        probs[idx] = c / shots
    return probs

# === 6. Define Loss
def loss(theta):
    probs = evaluate_circuit(theta)
    return np.mean((probs - target_probs)**2)

# === 7. เทรนด้วย classical optimizer
from scipy.optimize import minimize
np.random.seed(42)
initial_theta = np.random.uniform(0, np.pi, 8)

result = minimize(loss, initial_theta, method="COBYLA", options={"maxiter": 100})
trained_theta = result.x

# === 8. Generate synthetic samples
def sample_from_vqc(theta, n=300):
    probs = evaluate_circuit(theta, shots=n)
    samples = []
    for i, p in enumerate(probs):
        bit = format(i, f'0{n_qubits}b')
        samples += [bit] * int(p * n)
    return samples[:n]

bitstrings_generated = sample_from_vqc(trained_theta, n=300)

# === 9. Decode bitstring → Scaled features → Iris features
def decode_bits_to_features(bs):
    return [0.25 if b == '0' else 0.75 for b in bs]

def decode_all(bs_list, scaler):
    scaled = np.array([decode_bits_to_features(bs) for bs in bs_list])
    return scaler.inverse_transform(scaled)

iris_generated = decode_all(bitstrings_generated, scaler)

# === 10. บันทึกผล
df = pd.DataFrame(iris_generated, columns=["sepal length", "sepal width", "petal length", "petal width"])
df["label"] = "Generated by VQC"
df.to_csv("generated_iris.csv", index=False)
df.head()


,sepal length,sepal width,petal length,petal width,label
0,4.675,2.825,1.225,0.225,Generated by VQC
1,4.675,2.825,1.225,0.225,Generated by VQC
2,4.675,2.825,1.225,0.225,Generated by VQC
3,4.675,2.825,1.225,0.225,Generated by VQC
4,4.675,2.825,1.225,0.225,Generated by VQC
